In [1]:
import os
import csv
import time
import torch
from datasets import load_dataset
from google.cloud import translate_v2 as translate
from concurrent.futures import ThreadPoolExecutor    # Concurrent execution using threads
gcp_project_id = "artful-tractor-338209"   # Set the Google Cloud Project ID

C:\Users\karti\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\karti\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\Users\karti\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


## Create a Service Account:
After setting up project, enabling cloud translation api and setting billing account, In the Cloud Console, navigate to "IAM & Admin" > "Service accounts." Click "Create Service Account." Enter a name for service account, select the role(s) that service account needs (Cloud Translation API Editor), and click "Continue."

## Generate Key:
After creating the service account, click on it in the "Service accounts" page. Navigate to the "Keys" tab. Click on "Add Key" and choose "JSON." This will download a JSON key file containing the necessary credentials.

In [2]:
# give path to json key file

credential_path = "C:/Users/karti/Desktop/ComputerScience/internships/Nirant2/artful-tractor-338209-61f5638878ce.json"
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credential_path

In [3]:
possible_configs = [
    # "main",
    # "socratic",
    'ARC-Challenge',
    'ARC-Easy',
    # 'bbc hindi nli'
]
dataset = []
for config in possible_configs:
    dataset_slice = load_dataset("ai2_arc", config,ignore_verifications=True)
    dataset.append(dataset_slice)

C:\Users\karti\AppData\Local\Programs\Python\Python311\Lib\site-packages\datasets\load.py:1760: FutureWarning: 'ignore_verifications' was deprecated in favor of 'verification_mode' in version 2.9.1 and will be removed in 3.0.0.
You can remove this warning by passing 'verification_mode=no_checks' instead.
  warnings.warn(
Found cached dataset parquet (C:/Users/karti/.cache/huggingface/datasets/parquet/ARC-Challenge-6372c4cac1109a78/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)


  0%|          | 0/3 [00:00<?, ?it/s]

C:\Users\karti\AppData\Local\Programs\Python\Python311\Lib\site-packages\datasets\load.py:1760: FutureWarning: 'ignore_verifications' was deprecated in favor of 'verification_mode' in version 2.9.1 and will be removed in 3.0.0.
You can remove this warning by passing 'verification_mode=no_checks' instead.
  warnings.warn(
Found cached dataset parquet (C:/Users/karti/.cache/huggingface/datasets/parquet/ARC-Challenge-6372c4cac1109a78/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
# dataset
for g in dataset:
    print(g['train'])
    print(g['test'])
    print(g['validation'])

Dataset({
    features: ['id', 'question', 'choices', 'answerKey'],
    num_rows: 3370
})
Dataset({
    features: ['id', 'question', 'choices', 'answerKey'],
    num_rows: 3548
})
Dataset({
    features: ['id', 'question', 'choices', 'answerKey'],
    num_rows: 869
})
Dataset({
    features: ['id', 'question', 'choices', 'answerKey'],
    num_rows: 3370
})
Dataset({
    features: ['id', 'question', 'choices', 'answerKey'],
    num_rows: 3548
})
Dataset({
    features: ['id', 'question', 'choices', 'answerKey'],
    num_rows: 869
})


In [46]:
def translate_text( text: str,target='hi') -> dict:
    """Translates text into the target language.

    Target must be an ISO 639-1 language code.
    See https://g.co/cloud/translate/v2/translate-reference#supported_languages
    """

    translate_client = translate.Client()

    if isinstance(text, bytes):
        text = text.decode("utf-8")

    # Text can also be a sequence of strings, in which case this method
    # will return a sequence of results for each text.
    result = translate_client.translate(text, target_language=target)

    # html.unescape corrects translated symbols such as ', <,> etc.
    return html.unescape(result["translatedText"])
    
translate_text("How are you kartik")

'कार्तिक आप कैसे हैं?'

In [47]:
%%time

translate_text("{'text': ['dry palms', 'wet palms', 'palms covered with oil', 'palms covered with lotion'], 'label': ['A', 'B', 'C', 'D']}")

CPU times: total: 15.6 ms
Wall time: 937 ms


"{'पाठ': ['सूखी हथेलियाँ', 'गीली हथेलियाँ', 'तेल से ढकी हथेलियाँ', 'लोशन से ढकी हथेलियाँ'], 'लेबल': ['ए', 'बी', 'सी', 'डी' ']}"

In [8]:
num_cores = os.cpu_count()
print(f"Number of CPU cores: {num_cores}")

Number of CPU cores: 16


In [50]:
%%time

for i in range(len(possible_configs)):
    for set in dataset[i]:
        set_list = []

        # columns to translate
        columns = ['question','choices']

        # columns not to translate
        
        
        for col in columns:
            values = [str(item[col]) for item in dataset[i][set]]
            # print(values[1:2])
            
            # Use ThreadPoolExecutor for parallel translation
            if __name__ == '__main__':
                result =[]
                with ThreadPoolExecutor(max_workers=16) as exe:
                    # Maps the method with a list of values.
                    # result = list(exe.map(translate_text,values[1:2]))
                    
                    batch_size = 1
                    for j in range(0,len(values[:4]),batch_size):
                        j_end = min(j + batch_size, len(values[:4]))
                        res = list(exe.map(translate_text,values[j:j_end]))
                        # time.sleep(2)
                        result.extend(res)
                    
            set_list.append(result)
            print(result)

        # Create folders for each configuration
        current_directory = os.getcwd()
        
        # Specify the path of the 'config' folder
        config_folder_path = os.path.join(current_directory, possible_configs[i])
        
        # Create the 'config' folder
        os.makedirs(config_folder_path, exist_ok=True)       
    
        # Transpose the 2D list
        transposed_data = list(map(list, zip(*set_list)))
        path = os.path.join(possible_configs[i], f'{set}.csv')

        # append to previosly created csv file in case full dataset was not converted
        with open(path, 'a', encoding='utf-8') as f:
            # using csv.writer method from CSV package
            write = csv.writer(f)
            # write.writerow(columns)
            write.writerows(transposed_data)

['जॉर्ज अपने हाथों को जल्दी-जल्दी रगड़कर गर्म करना चाहता है। त्वचा की कौन सी सतह सबसे अधिक ऊष्मा उत्पन्न करेगी?', 'निम्नलिखित में से कौन सा कथन सबसे अच्छी तरह से बताता है कि चुंबक आमतौर पर रेफ्रिजरेटर के दरवाजे से क्यों चिपकते हैं?', 'तलछटी चट्टान की परतों में देखी गई एक तह संभवतः इसी के परिणामस्वरूप हुई है', 'मेसोज़ोइक युग के अंत में कई पौधे और जानवर क्यों मर गए, इसके लिए वैज्ञानिक इनमें से सबसे हालिया स्पष्टीकरण क्या देते हैं?']
["{'पाठ': ['सूखी हथेलियाँ', 'गीली हथेलियाँ', 'तेल से ढकी हथेलियाँ', 'लोशन से ढकी हथेलियाँ'], 'लेबल': ['ए', 'बी', 'सी', 'डी' ']}", "{'टेक्स्ट': ['रेफ्रिजरेटर का दरवाजा चिकना है।', 'रेफ्रिजरेटर के दरवाजे में लोहा है।', 'रेफ्रिजरेटर का दरवाजा एक अच्छा कंडक्टर है।', 'रेफ्रिजरेटर के दरवाजे में बिजली के तार हैं।'], 'लेबल ': ['ए बी सी डी']}", "{'पाठ': ['बहते मैग्मा का ठंडा होना।', 'क्रस्टल प्लेटों का अभिसरण', 'नदी तलछट का जमाव।', 'कार्बोनेट खनिजों का समाधान।'], 'लेबल': ['ए', 'बी' ', 'सी', 'डी']}", "{'पाठ': ['विश्वव्यापी बीमारी', 'वैश्विक पर्वत निर्माण', 'पौधों और जा